In [1]:
# 2018-09-20
# A. Pendleton
# Link gene models from version 3 (500kb introns) to version 2 (100 kb introns)

In [53]:
#this uses iPython magic to make plots appear inline
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
import sys
import numpy as np
import matplotlib.patches as patches
import gzip
import fileinput
import glob
from scipy import stats
import re
from matplotlib_venn import venn3, venn3_circles
from collections import OrderedDict


def count_lines(f):
    lineCount = 0
    with open(f, 'r') as f:
        for line in f:
            lineCount += 1
        return lineCount
def runCMD(cmd):
    #print(cmd)
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)
# TO REMOVE TOP AND RIGHT AXIS OF PLOTS
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

# Define inputs

In [4]:
v2Dir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/NonRedundant_NoRMIntersect_FilteredGeneSet/'
v2CDSFile = v2Dir + 'TotalSet_NoRMSingleExons_AllMultiExons_cds.fa'

v3Dir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/NonRedundant_NoRMIntersect_FilteredGeneSet/'
v3CDSFile = v3Dir + 'TotalSet_NoRMSingleExons_AllMultiExons_cds.fa'


# Set working directory for linking v2 to v3

In [6]:
wkDir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/LinkToVersion2Models/'

# BLAT commands

In [8]:
# fatotwobit version2 to make database
cmd = 'faToTwoBit %s %s.2bit' % (v2CDSFile,v2CDSFile)
print(cmd)
#runCMD(cmd)

#BLAT
blatFile = wkDir + 'BLAT_version3_against_version2.psl' #desired output file
cmd = 'blat %s.2bit %s %s' % (v2CDSFile,v3CDSFile,blatFile)
print(cmd)
#runCMD(cmd)


faToTwoBit /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/NonRedundant_NoRMIntersect_FilteredGeneSet/TotalSet_NoRMSingleExons_AllMultiExons_cds.fa /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/NonRedundant_NoRMIntersect_FilteredGeneSet/TotalSet_NoRMSingleExons_AllMultiExons_cds.fa.2bit
blat /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/NonRedundant_NoRMIntersect_FilteredGeneSet/TotalSet_NoRMSingleExons_AllMultiExons_cds.fa.2bit /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/NonRedundant_NoRMIntersect_FilteredGeneSet/TotalSet_NoRMSingleExons_AllMultiExons_cds.fa /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/LinkToVersion2Models/BLAT_version3_against_version2.psl


# Create list of all version 3 (500kb intron) genes

In [37]:
v3Genes = []

for line in open(v3CDSFile,'r'):
    if '>' not in line: #only want fasta headers which have the gene IDs in them
        continue
    line = line.rstrip()
    line = line.replace('>','') #remove > symbol
    
    v3Genes.append(line)
print('%i v3 genes (500kb introns) added to dictionary' % len(v3Genes))

45443 v3 genes (500kb introns) added to dictionary


# Parse BLAT file to link the gene models

In [23]:
linkDict = {}

for line in open(blatFile,'r'):
    line = line.rstrip().split()
    #skip header lines
    if 'psLayout' in line or len(line) == 0 or 'match' in line[0] or '-' in line[0]:
        continue
    
    hitScore = int(line[0])
    strand = line[8]
    
    v3Gene = line[9] # QUERY
    v3Length,v3Start,v3End = int(line[10]),int(line[11]),int(line[12])

    v2Gene = line[13] # DATABASE GENE
    v2Length,v2Start,v2End = int(line[14]),int(line[15]),int(line[16])
    
    ####FILTRATIONS
    #BLOCK COUNT
    blockCount = int(line[17])
    if blockCount != 1:
        continue
    
    #Must be same length
    if v3Length != v2Length:
        continue
    
    #Score must equal length of both genes
    if hitScore != v3Length:
        continue
    
    linkDict[v3Gene] = v2Gene
    #print(line) 
    #break

print('%i v3 genes (500kb introns) could be linked to v2 genes' % len(linkDict.keys()))

22209 v3 genes (500kb introns) could be linked to v2 genes


# Determine which v3 genes do not have a match in v2

In [40]:
linkFile = open(wkDir + 'Link_Version3Genes_Version2Genes.txt','w')
hasPairing, noPairing = 0, 0


for gene in v3Genes:
    if gene not in linkDict.keys():
        linkFile.write('%s\tNA\n' % (gene))
        noPairing+=1
    else:
        linkFile.write('%s\t%s\n' % (gene,linkDict[gene]) )
        hasPairing+=1
linkFile.close()

print('%i/%i genes have a match in version 2' % (hasPairing,len(v3Genes)))
print('%i/%i genes DO NOT have a match in version 2' % (noPairing,len(v3Genes)))
    

22209/45443 genes have a match in version 2
23234/45443 genes DO NOT have a match in version 2


# BLAST

### Define inputs

In [63]:
blastDir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/BLAST2GO/'

cmd = 'mkdir -p %sinput/' % blastDir
print(cmd)
runCMD(cmd)


#Create directory for split fastas
fastaOutDir = blastDir + 'input/' + 'split_fastas/'
cmd = 'mkdir -p %s' % fastaOutDir
print(cmd)
runCMD(cmd)

#Out directory for BLAST results
blastpDir = blastDir + 'blastp_results/'
cmd = 'mkdir -p %s' % blastpDir
print(cmd)
runCMD(cmd)

#Scripts directory to write all the blastp commands to
scriptsDir = blastDir + 'scripts/'
cmd = 'mkdir -p %s' % scriptsDir
print(cmd)
runCMD(cmd)

mkdir -p /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/BLAST2GO/input/
mkdir -p /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/BLAST2GO/input/split_fastas/
mkdir -p /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/BLAST2GO/blastp_results/
mkdir -p /home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/rna-seq/version3_intron500k/BLAST2GO/scripts/


### Split peptide fasta file into individual fasta files

In [ ]:
#FASTA information
fastaFile = v3CDSFile.replace('.cds','.pep')

commandsfile = scriptsDir + 'blastp_commands.cmds'
commandsFile = open(commandsfile, 'w')

sequenceCount = 0
copied,wroteCommand=0,0

for line in open(fastaFile,'r'):
    line = line.rstrip() 
    if '>' in line and sequenceCount > 0: #close the fasta file if it's the next fasta seq
        fastaOut.close()

    if '>' in line: #create new fasta file
        sequenceCount += 1 #add one to sequence counts
        #format so that the gene ID doesn't have "ID=" or anything after 
        #         the first semi colon or else blastp wont work
        geneID = line.replace('>','') 
        if 'ID=' in geneID:
            geneID = geneID.replace('ID=','')

        #Creating fasta out file just with one sequence
        fasta_out = fastaOutDir + geneID + '.fa'
        fastaOut = open(fasta_out, 'w')
        fastaOut.write('>%s\n' % geneID)  #writes the gene ID line only here

        ###ONLY WRITE BLASTP COMMAND IF NOT ALREADY DONE IN VERSION 2
        if geneID in linkDict.keys(): #if gene has mate in version 2
            originalBLASTpDir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/BLAST2GO/version2_intron100k/blastp_results/'
            #Copy previous XML file to version3 directory
            ## AND
            ##Rename to the version 3 gene
            cmd = 'cp %s%s.xml %s%s.xml' % (originalBLASTpDir,linkDict[geneID],blastpDir,geneID)
            runCMD(cmd)
            copied+=1
        
        else:#WRITE COMMAND TO COMMAND FILE since gene wasn't blasted previously
            cmd = 'blastp -db ~/links/kidd-lab-scratch/blastdb/nr -outfmt 5 -evalue 1e-3 -word_size 3 -show_gis -max_hsps_per_subject 20 -num_threads 5 -max_target_seqs 20 -out %s%s.xml -query %s' % (blastpDir, geneID, fasta_out)
            commandsFile.write('%s\n' % cmd)
            wroteCommand +=1

    else:
        fastaOut.write('%s\n' % line)


fastaOut.close()
commandsFile.close()

print ('Processed %d fasta sequences' % sequenceCount)
print('Copied xml files from %i sequences' % copied)
print('Wrote commands for %i sequences' % wroteCommand)
